In [1]:
%matplotlib inline
# %matplotlib notebook
%config Completer.use_jedi = False
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.special import logsumexp
from scipy.special import softmax
import numpy.linalg as linalg
from sklearn.cluster import KMeans
from sklearn import metrics
import scipy.sparse as sparse


In [2]:
random_seed = 123
rng = np.random.default_rng(random_seed)

# 1 Import Dataset

In [3]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', parser ='auto')

Binarize the Data

In [4]:
X_mnist = np.array(mnist.data) > 0.5 # binary-rise the data
Y_mnist = np.array(mnist.target, dtype ='int')
X_mnist_train = X_mnist[:4000, :] # use the first 4000 images as training data
Y_mnist_train = Y_mnist[:4000]
X_mnist_test = X_mnist[4000:5000, :] # the next 1000 as testing
Y_mnist_test = Y_mnist[4000:5000]

# 2 Algorithm

## 2.1 Expectation Step 

Compute responsibility (probability of component generating datapoint) :
$$\begin{align}
\ln r_{ik} &= \ln\pi_k + \ln p(\mathbf{x}^{(i)}| z^{(i)}=k, \{\boldsymbol{\mu}_k\}) + C\\
&=\ln\pi_k + \sum_{d=1}^D x_d^{(i)} \ln\mu_{kd} + (1-x_d^{(i)}) \ln(1-\mu_{kd}) + C
\end{align}$$


Log Likelihood:
$$\ell(\boldsymbol{\pi}, \boldsymbol{\mu}s)=\frac{1}{n}\ln p(\{\mathbf{x}^{(i)}\}_{i=1}^n|\boldsymbol{\pi}, \{\boldsymbol{\mu}_k\}) = \frac{1}{n} \sum_{i=1}^n\ln\left \{ \sum_{k=1}^K \pi_k \cdot p(\mathbf{x}^{(i)}| \boldsymbol{\mu}_k)\right \}$$

In [5]:
def e_step(X, πs, μs, eps=1e-14):
    '''
        πs : 3 x 1 (priors)
        μs : 3 x 9 (component means for each pixel)
        X  : 500 x 9
    '''
    n, D = X.shape
    K = len(πs)
    R = np.zeros((n, K))
    loglik = 0.0

    for i in range(0,n):
        comp1 = np.dot(X[i], np.log(μs + eps).T)
        comp2 = np.dot(X[i], np.log(1 - μs + eps).T)
        sum = np.log(πs + eps) + comp1 + comp2

        R[i] = sum  

    # Compute the norm for each observation 
    log_R_norm = logsumexp(R, axis=1, keepdims=True)

    # Normalize R to sum to 1 across the clusters for each observation 
    R = np.exp(R - log_R_norm)
    
    # Compute average marginal log likelihood 
    loglik = np.sum(log_R_norm) / n        

    return R, loglik

# 2.2 Maximization Step

For $k=1, \ldots, K$:

$$\pi_k = \frac{n_k}{n};\;\; \boldsymbol{\mu}_k = \frac{\sum_{i=1}^n r_{ik} \cdot \mathbf{x}^{(i)}}{n_k},$$


* where $n_k = \sum_{i=1}^n r_{ik}$

In [ ]:
def m_step(X, R):
    ## implement here 
    n, d = X.shape # rows are each datapoint, cols are responsibility to each of K clusters

    nk = np.sum(R, axis=0) 
    πs = nk/n
    μs = np.dot(R.T, X)/nk[:, np.newaxis]

    return πs, μs